In [138]:
import yfinance as yf
import pandas as pd
from datetime import datetime, timedelta
import pytz

In [ ]:
aapl = yf.download(tickers="AAPL", period="7d", interval="1m")
aapl

In [ ]:
data=pd.read_csv(r'C:\Users\Legion\Desktop\FinalYearProject\News.csv')
data

,Date,article_title,article,source_name,source_link,Positive,Negative,Neutral,Compound,Sentiment
0,2024-03-28 15:22:42+00:00,Magnificent Seven Stocks To Buy And Watch: App...,"Magnificent Seven stocks, including AI leader ...",Yahoo Finance,https://finance.yahoo.com/quote/AAPL/m/4205eaa...,0.310,0.000,0.690,0.8442,Positive
1,2024-03-28 14:49:00+00:00,"Apple-Supplier Foxconn Hits Record High, and O...",Taiwan's Foxconn hit a record closing high on ...,Yahoo Finance,https://finance.yahoo.com/quote/AAPL/m/d8d8576...,0.162,0.000,0.838,0.6808,Positive
2,2024-03-28 14:00:01+00:00,Apple: A Compounding Machine Slowing Down,Apple Inc. (NASDAQ:AAPL) is on the podium for ...,Yahoo Finance,https://finance.yahoo.com/quote/AAPL/news/appl...,0.153,0.037,0.810,0.9996,Positive
3,2024-03-28 13:57:33+00:00,Dow Jones Rises After Surprise GDP; Apple Slid...,Stock Market Today: The Dow Jones rose Thursda...,Yahoo Finance,https://finance.yahoo.com/quote/AAPL/m/f450db0...,0.104,0.000,0.896,0.2732,Positive
4,2024-03-28 13:38:43+00:00,Can Apple Rise 17%? This Option Trade Offers S...,Selling this spread would generate roughly $75...,Yahoo Finance,https://finance.yahoo.com/quote/AAPL/m/8263281...,0.000,0.000,1.000,0.0000,Neutral
...,...,...,...,...,...,...,...,...,...,...
539,2024-03-05 16:59:20+00:00,Apple Stock Gloom Deepens as Pressure to Show ...,(Bloomberg) -- Apple Inc.’s move to shutter it...,Yahoo Finance,https://finance.yahoo.com/quote/AAPL/news/appl...,0.055,0.051,0.894,-0.0880,Negative
540,2024-03-05 16:55:49+00:00,What the shakeup in Magnificent 7 stocks means...,Stock market indices have been propped up on t...,Yahoo Finance,https://finance.yahoo.com/quote/AAPL/video/sha...,0.118,0.028,0.854,0.9988,Positive
541,2024-03-05 16:46:00+00:00,Alphabet (GOOGL) Enhances Google Maps With New...,Alphabet’s GOOGL Google is gaining strong mome...,Yahoo Finance,https://finance.yahoo.com/quote/AAPL/news/alph...,0.171,0.000,0.829,0.9982,Positive
542,2024-03-05 16:43:00+00:00,"Why Apple Stock Keeps Dropping, and What Could...","Apple stock is in the red this year, so Wall S...",Yahoo Finance,https://finance.yahoo.com/quote/AAPL/m/c76aec8...,0.000,0.038,0.962,-0.4767,Negative


In [ ]:
def convertTimezone(timezone, df,name):
    """
    Converting timestamps in to a specified timezone.

    Arguments:
        timezone (str): Target timezone to convert timestamps to.
        df : DataFrame containing a column of timestamp strings.
        name: name of the timestamp column

    Returns a dataFrame with timestamps converted to the specified timezone.
    """
    # Convert timestamp column to datetime objects
    df[name] = pd.to_datetime(df[name])

    # Create timezone object for the target timezone
    target_tz = pytz.timezone(timezone)

    # Convert timestamps to the target timezone
    df[name] = df[name].dt.tz_convert(target_tz)

    return df

    

In [ ]:
hist_df = convertTimezone('UTC', data,'Datetime')
hist_df

,Datetime,Open,High,Low,Close,Adj Close,Volume
0,2024-03-05 14:30:00+00:00,171.000000,171.000000,170.610001,170.919998,170.919998,5958861
1,2024-03-05 14:31:00+00:00,171.020004,171.074997,171.020004,171.059998,171.059998,926341
2,2024-03-05 14:32:00+00:00,171.335999,171.339996,171.270004,171.300003,171.300003,703908
3,2024-03-05 14:33:00+00:00,171.929993,171.970001,171.875000,171.895004,171.895004,994237
4,2024-03-05 14:34:00+00:00,171.625000,171.740005,171.580002,171.610001,171.610001,626087
...,...,...,...,...,...,...,...
1914,2024-03-11 19:55:00+00:00,172.570007,172.675003,172.464996,172.664993,172.664993,306481
1915,2024-03-11 19:56:00+00:00,172.660004,172.779999,172.625000,172.770004,172.770004,317942
1916,2024-03-11 19:57:00+00:00,172.770004,172.839996,172.720001,172.809998,172.809998,349009
1917,2024-03-11 19:58:00+00:00,172.800003,172.807800,172.679993,172.729996,172.729996,408144


In [ ]:
# get specific stock price at a certain day and time
# it will take as parameters the date of the published article, the number of minutes after the article was published

# i need to make sure that the stock prices dataset has no index.
# and i need to convert its timezone to UTC
# i need to make sure that the end date is 1 day ahead.



In [ ]:
# label the data

In [ ]:
def getStockPrice(ticker,date):
    # date_string = '2024-03-05 14:30:00+00:00'
    # format = '%Y-%m-%d %H:%M:%S%z'
    # date = datetime.strptime(date_string, format) # to pass as an argument

    end_date=date+timedelta(days=1)
    end_date=end_date.strftime("%Y-%m-%d %H:%M:%S%z").split()
    end_date=end_date[0]

    date= date.strftime("%Y-%m-%d %H:%M:%S%z").split()
    day=date[0]
    time=date[1].split("+")[0]

    data = yf.download(tickers=ticker, start=day, end=end_date, interval='1m', index_col=0)
    
    if data.empty:
        print(f"No data available for {ticker} on {date}")

    data_at_time = data.between_time(time, time)

    if not data_at_time.empty:
        stock_price = data_at_time['Close'].values[0]

    else:
        print(f"No data available for {ticker} on {date} at {time}")
    
    
    if stock_price is not None:
        print(f"The stock price of {ticker} on {day} at {time} was ${stock_price:.2f}")

In [ ]:
# testing if the function works:
ticker = 'AAPL'
date = datetime(2024, 3, 5, 14, 30, 0)  # Assuming date is in UTC timezone with offset
getStockPrice(ticker, date)

In [ ]:
ticker = 'AAPL'
date = datetime(2024, 3, 5, 14, 35, 0)  # Assuming date is in UTC timezone with offset
getStockPrice(ticker, date)